## 讀取選定文章，2016/1-2018/6當作訓練資料集，2018/7~2018/12當作測試資料集，去預測

In [1]:
import csv
import pandas as pd
import numpy as np
import jieba
import jieba.analyse

In [2]:
df_tgg=pd.read_csv("bda2019_dataset/MTK_stock.csv", encoding="utf-8")
df_tgg=df_tgg.drop(columns=['Unnamed: 0'])
df_tgg.head(5)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,highpriceshift,rate,label
0,2454 聯發科,2018-12-25,221.0,222.0,219.0,221.0,"1,528","337,471","1,229","1,591,506",12.90,1.29,220.0,-0.004525,平
1,2454 聯發科,2018-12-24,223.5,225.5,222.5,225.0,"1,674","375,607","1,173","1,591,506",13.13,1.32,221.0,-0.017778,跌
2,2454 聯發科,2018-12-22,223.0,223.5,221.5,223.0,923,"205,274",860,"1,591,506",13.02,1.31,225.0,0.008969,平
3,2454 聯發科,2018-12-21,224.0,226.0,221.5,223.5,"5,269","1,177,829","2,560","1,591,506",13.05,1.31,223.0,-0.002237,平
4,2454 聯發科,2018-12-20,229.5,231.0,223.5,225.0,"9,042","2,049,881","5,840","1,591,506",13.13,1.32,223.5,-0.006667,平


In [8]:
tnews=pd.read_csv("bda2019_dataset/MTK_content.csv", encoding="utf-8")
tnews=tnews.drop(columns=['Unnamed: 0'])
tnews['post_time']=pd.to_datetime(tnews['post_time'])  #把日期轉成datefime格式
tnews.head(5)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label,cutcontent
0,1451783510043_N01,news,yahoo股市,重大要聞,2016-01-03 08:58:00,《各報要聞》30檔電子股，去年可賺1股本,時報資訊,景氣不佳，電子業成了去年放無薪假的大戶，但在一片不景氣聲中，還是有科技公司逆流而上，初估20...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,景氣 不佳 電子業 成 了 去年 放無薪假 的 大戶 但 在 一片 不景氣 聲中 ...
1,1451874408573_N01,news,yahoo股市,重大要聞,2016-01-04 09:59:00,台股開紅盤重摔 CES概念撐腰 IC設計鈺創、凌通、迅杰大漲撐盤,鉅亨網,台股今(4)日2016年第一個交易日，開盤不久指數就面臨賣壓襲擊，摜破8200點與月線支撐，...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,台股 今 ( 4 ) 日 2016 年 第一個 交易日 開盤 不久 指數 就 面臨 賣壓...
2,1451952709026_N01,news,yahoo股市,重大要聞,2016-01-05 07:48:00,《金融》曾銘宗：金融業去年獲利可望再創高,時報資訊,金管會昨（4）日舉辦新春記者會，主委曾銘宗表示，若加計銀行針對中國曝險及不動產的增提準備，2...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,金管會 昨 （ 4 ） 日 舉辦 新春 記者會 主委 曾 銘宗 表示 若 加計 銀行...
3,1451968008054_N01,news,yahoo股市,重大要聞,2016-01-05 12:05:00,台股盤中－TRF風暴 止穩之路遭金融股扯後腿 8100苦戰,鉅亨網,美股昨天重挫但留下200多點下影線，台股今(5)日開低，最低摔到8072點，一度跌破8100...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,美股 昨天 重挫 但 留下 200 多點 下影線 台股 今 ( 5 ) 日 開低 最...
4,1451978807822_N01,news,yahoo股市,重大要聞,2016-01-05 15:07:00,【Y晚報】沙伊交惡！油價只是死貓跳 專家唱衰剩1字頭,Yahoo奇摩股市,美股道瓊昨天連續第三天下挫，收盤大跌276點，不過尾盤拉高收斂跌幅，留下191點的下影線，因...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,美股 道瓊 昨天 連續 第三天 下挫 收盤 大跌 276 點 不過 尾盤 拉高 收斂...


In [9]:
#取前多少個keyword
keywords=list()
with open('bda2019_dataset/companykeyword3000.txt', 'r',encoding='utf-8') as data:
    for keyword in data:
        keywords.append(keyword.strip('\n'))
keywords=keywords[:1500]

In [11]:
#把2018-7-1以前的資料設為train
#把2018-7-1以後的資料設為test
tnews_train=tnews[(tnews['post_time']>='2016-1-1 00:00:00') & (tnews['post_time']<'2018-7-1 00:00:00')]
tnews_test=tnews[tnews['post_time']>='2018-7-1 00:00:00']
print(len(tnews_train),len(tnews_test))

3402 1733


In [12]:
#train的資料比例
tnews1=tnews_train[tnews_train['label'].str.contains('漲')]
tnews2=tnews_train[tnews_train['label'].str.contains('跌')]
tnews3=tnews_train[tnews_train['label'].str.contains('平')]
print(len(tnews_train),len(tnews1),len(tnews2),len(tnews3))

3402 1017 869 1516


In [13]:
#test資料比例
tnews1=tnews_test[tnews_test['label'].str.contains('漲')]
tnews2=tnews_test[tnews_test['label'].str.contains('跌')]
tnews3=tnews_test[tnews_test['label'].str.contains('平')]
print(len(tnews_test),len(tnews1),len(tnews2),len(tnews3))

1733 576 530 627


#  train資料處理

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv=TfidfVectorizer(vocabulary=keywords) # 預設用只要有空格就一個欄位
# cv=TfidfVectorizer(vocabulary=all0431) # 預設用只要有空格就一個欄位
tfidf=pd.SparseDataFrame(cv.fit_transform(tnews_train['cutcontent']), 
                       tnews_train.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)
tfidf.head(1)
tfidf=tfidf.drop(columns=['聯發科'])

In [15]:
tfidf['煞氣的label']=tnews_train['label']
tfidf2=tfidf.drop(columns=['煞氣的label'])
len(tfidf)

3402

In [16]:
from sklearn import model_selection
X_train = tfidf2.to_numpy()
Y_train = tfidf['煞氣的label'].to_numpy()

#  test資料處理

In [17]:
tfidf3=pd.SparseDataFrame(cv.fit_transform(tnews_test['cutcontent']), 
                       tnews_test.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)
tfidf3=tfidf3.drop(columns=['聯發科'])

In [18]:
tfidf3['煞氣的label']=tnews_test['label']
tfidf4=tfidf3.drop(columns=['煞氣的label'])
len(tfidf3)

1733

In [19]:
X_test=tfidf4.to_numpy()
Y_test=tfidf3['煞氣的label'].to_numpy()

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [21]:
#SVM
from sklearn.svm import SVC

SVC_model = SVC(kernel='linear')
SVC_model.fit(X_train,Y_train)
predictions = SVC_model.predict(X_test)

print(SVC_model.score(X_train,Y_train))
print(accuracy_score(Y_test ,predictions))
print(confusion_matrix(Y_test, predictions))
print(classification_report(Y_test, predictions))

0.6540270429159318
0.3387189844200808
[[400 195  32]
 [396 141  39]
 [345 139  46]]
              precision    recall  f1-score   support

           平       0.35      0.64      0.45       627
           漲       0.30      0.24      0.27       576
           跌       0.39      0.09      0.14       530

   micro avg       0.34      0.34      0.34      1733
   macro avg       0.35      0.32      0.29      1733
weighted avg       0.35      0.34      0.30      1733



In [22]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier

decision_model = DecisionTreeClassifier()
decision_model.fit(X_train,Y_train)
predictions1 = decision_model.predict(X_test)

print(decision_model.score(X_train, Y_train))
print(accuracy_score(Y_test, predictions1))
print(confusion_matrix(Y_test, predictions1))
print(classification_report(Y_test, predictions1))

0.9920634920634921
0.3150605885747259
[[275 213 139]
 [275 170 131]
 [272 157 101]]
              precision    recall  f1-score   support

           平       0.33      0.44      0.38       627
           漲       0.31      0.30      0.30       576
           跌       0.27      0.19      0.22       530

   micro avg       0.32      0.32      0.32      1733
   macro avg       0.31      0.31      0.30      1733
weighted avg       0.31      0.32      0.31      1733



In [23]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification

forest_model=RandomForestClassifier()
forest_model.fit(X_train,Y_train)
predictions2=forest_model.predict(X_test)

print(forest_model.score(X_train,Y_train))
print(accuracy_score(Y_test, predictions2))
print(confusion_matrix(Y_test, predictions2))
print(classification_report(Y_test, predictions2))

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9779541446208113
0.34160415464512406
[[410 145  72]
 [399 127  50]
 [363 112  55]]
              precision    recall  f1-score   support

           平       0.35      0.65      0.46       627
           漲       0.33      0.22      0.26       576
           跌       0.31      0.10      0.16       530

   micro avg       0.34      0.34      0.34      1733
   macro avg       0.33      0.33      0.29      1733
weighted avg       0.33      0.34      0.30      1733



In [24]:
#MLP
from sklearn.neural_network import MLPClassifier

model_MLP = MLPClassifier(hidden_layer_sizes=(200,))
model_MLP.fit(X_train, Y_train)
print(model_MLP.score(X_train,Y_train))

predictions = model_MLP.predict(X_test)
print(accuracy_score(Y_test, predictions))
print(confusion_matrix(Y_test, predictions))
print(classification_report(Y_test, predictions))

0.9888300999412111
0.3190998268897865
[[277 227 123]
 [266 183 127]
 [241 196  93]]
              precision    recall  f1-score   support

           平       0.35      0.44      0.39       627
           漲       0.30      0.32      0.31       576
           跌       0.27      0.18      0.21       530

   micro avg       0.32      0.32      0.32      1733
   macro avg       0.31      0.31      0.31      1733
weighted avg       0.31      0.32      0.31      1733



/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
